# Indeed.com scraping with Selenium

In [53]:
#import packages
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
from selenium import webdriver
import time
from datetime import datetime

- Delete the job title and location in the url and replace it with curly braces. Create a base_url that you'll use to create whatever job position or location that you want.

In [55]:
base_url = 'https://nl.indeed.com/jobs?q={}&l={}&start='

- Create a function called get_url, with the arguments of position and location. Then move the base_url variable inside this function.

In [56]:
def get_url(position, location):
    """Generate a url from position and location"""
    base_url = 'https://nl.indeed.com/jobs?q={}&l={}&start='
    url = base_url.format(position, location)
    return url

In [57]:
url = get_url('data analist', 'Nederland')

In [59]:
url

'https://nl.indeed.com/jobs?q=data analist&l=Nederland&start='

In [60]:
# create driver object
driver = webdriver.Chrome()
time.sleep(10)

### Access the site programmatically

- Tell the browser to visit the indeed page. Call the driver object created above and use the get method; pass the URL of the website you'd like to extract.

In [61]:
# asses webpage
driver.get(url)

- Accept cookies, if applicable.

In [62]:
try:
    driver.find_element_by_id('onetrust-accept-btn-handler').click()
    time.sleep(2)
except:
    print('probably accepted the cookie already!')

- Use BeautifulSoup; create the res object from the driver object.

In [63]:
res = driver.page_source.encode('utf-8')
soup = BeautifulSoup(res, "html.parser")

In [64]:
# store job card links in list
jobs = driver.find_elements_by_class_name('jobsearch-SerpJobCard')
#check
jobs

[<selenium.webdriver.remote.webelement.WebElement (session="a515bb6b450de5b70aa46c1ac3eb487e", element="0.31923624151550656-2")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a515bb6b450de5b70aa46c1ac3eb487e", element="0.31923624151550656-3")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a515bb6b450de5b70aa46c1ac3eb487e", element="0.31923624151550656-4")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a515bb6b450de5b70aa46c1ac3eb487e", element="0.31923624151550656-5")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a515bb6b450de5b70aa46c1ac3eb487e", element="0.31923624151550656-6")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a515bb6b450de5b70aa46c1ac3eb487e", element="0.31923624151550656-7")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a515bb6b450de5b70aa46c1ac3eb487e", element="0.31923624151550656-8")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a515bb6b450de5b70aa46c1ac3eb48

## Loop through the pages and collect the ids from the current page

In [157]:
jobids = []

# loop through the pages
for i in range(0,800,10):
    driver.get(url+str(i))
    time.sleep(6)
    jobs = driver.find_elements_by_class_name('jobsearch-SerpJobCard')
    
    # collect ids from current page
    for job in jobs:
        jobid = job.get_attribute('data-jk')
        jobids.append(jobid)

In [158]:
#check
jobids[:20]

['a9749dd1260d0ccc',
 '45f4c626b11ee6dd',
 '5c387427784186c2',
 '7f8fe03c5484706b',
 '8f24ece2c83af849',
 '2b6766d1d03caa6c',
 '275eaed96278f6a1',
 '259778b2faa5e0af',
 'fea9b49357181e96',
 '9b8b8e70f8af0681',
 '6f88c3b37ffc3fad',
 '5595e58ddc3344ca',
 '42d2dbc0740f816f',
 '07b0178cd8c65029',
 '83cd6f19ccc27967',
 '83cd6f19ccc27967',
 'f20e3fbd9eef4ad7',
 '9e7363bfa414ca82',
 '9237811ad0473c04',
 '009180d11b2d5df0']

### Get the job title

In [84]:
driver.get('https://nl.indeed.com/vacature-bekijken?jk=024641b602f7096c')
time.sleep(5)
title = driver.find_element_by_class_name('jobsearch-JobInfoHeader-title').text
#check
title

'Data Analist'

### Get the job company

In [85]:
driver.get('https://nl.indeed.com/vacature-bekijken?jk=024641b602f7096c')
time.sleep(5)
company = driver.find_element_by_class_name('jobsearch-InlineCompanyRating').text
#check
company

'Aon'

### Get the job salary

In [90]:
driver.get('https://nl.indeed.com/vacature-bekijken?jk=024641b602f7096c')
time.sleep(5)
try:
    salary = driver.find_element_by_class_name('jobsearch-JobMetadataHeader-item').text
except:
    salary = ""
#check
salary

'€2.600 - €3.500 per maand'

### Get the job location

In [114]:
driver.get('https://nl.indeed.com/vacature-bekijken?jk=024641b602f7096c')
time.sleep(5)
location = driver.find_element_by_class_name('jobsearch-DesktopStickyContainer-subtitle').text.splitlines()[1]
#check
location

'Postcode 5651 in Eindhoven'

### Get the job company review

In [100]:
driver.get('https://nl.indeed.com/vacature-bekijken?jk=024641b602f7096c')
time.sleep(5)
companyreview = driver.find_element_by_class_name('jobsearch-CompanyReview').text.replace('\n',' ')
#check
companyreview

'Aon 4,610 reviews Lees wat andere gebruikers over dit bedrijf te zeggen hebben.'

### Get the job description

In [118]:
driver.get('https://nl.indeed.com/vacature-bekijken?jk=024641b602f7096c')
time.sleep(5)
description = driver.find_element_by_id('jobDescriptionText').text.replace('\n',' ')
#check
description

'Over deze vacature In het kort Ben je analytisch sterk en altijd op zoek naar mogelijkheden om te helpen bij het verwezenlijken van bedrijfsdoelstellingen? Wil jij continu het beste halen uit data en met het team om tafel voor de beste strategie? Dan is de functie van Data Analist zeker iets voor jou! Jij brengt mee Minimaal hbo werk- en denkniveau Sterke affiniteit met processen en IT Grote mate van zelfstandigheid en daadkracht Je vindt bij ons Vaste 13e maand Bonusregeling Premievrij pensioen Tom von Wolzogen Kühr Recruiter 06 22581665  tom.von.wolzogen.kuhr@aon.com  linkedin.com/in/tomvonwolzogenkühr Wat ga je doen? Met de toenemende groei van data is de behoefte naar goede informatie en het gebruik en interpreteren daarvan explosief gestegen. Data is dan ook één van de internationale pijlers binnen Aon. Als Data Analist krijg je de ruimte om te pionieren. Je geeft proactief inzicht in het gedrag van onze klanten, identificeer en kwantificeer je risico’s, signaleer je trends en on

### Get the job posting date

In [122]:
driver.get('https://nl.indeed.com/vacature-bekijken?jk=024641b602f7096c')
time.sleep(5)
postingdate = driver.find_element_by_class_name('jobsearch-JobMetadataFooter').text.splitlines()[1]
#check
postingdate

'30+ dagen geleden'

# Putting it all together

- Note: if it asks you at some point to prove that you're not a robot, copy the link in a new window and select the right images.
- Tip for the the next trial: separate the ids in multiple bundles and run them separately to ensure that it doens't crush.

In [159]:
titles = []
companies = []
locations = []
salaries = []
postingdates = []
companyreviews = []
descriptions = []
#urls = []

# Use jobids2 = jobids, because you can run it for a selected segment, by setting jobids to, for example, jobids[:3] for the first 3 elements. And so, you only have to change this once, here.
jobids2 = jobids

for jobid in jobids2:
    driver.get('https://nl.indeed.com/vacature-bekijken?jk='+jobid)
    time.sleep(5)
    try:
        titles.append(driver.find_element_by_class_name('jobsearch-JobInfoHeader-title').text)
    except:
        titles.append('')
    try:
        companies.append(driver.find_element_by_class_name('jobsearch-InlineCompanyRating').text)
    except:
        companies.append('')
    try:
        locations.append(driver.find_element_by_class_name('jobsearch-DesktopStickyContainer-subtitle').text.splitlines()[1])
    except:
        locations.append('')
    try:
        salaries.append(driver.find_element_by_class_name('jobsearch-JobMetadataHeader-item').text)
    except:
        salaries.append('')
    try:
        postingdates.append(driver.find_element_by_class_name('jobsearch-JobMetadataFooter').text.splitlines()[1])
    except:
        postingdates.append('')
    try:
        companyreviews.append(driver.find_element_by_class_name('jobsearch-CompanyReview').text.replace('\n',' '))
    except:
        companyreviews.append('')
    try:
        descriptions.append(driver.find_element_by_id('jobDescriptionText').text.replace('\n',' '))
    except:
        descriptions.append('')
    
    #To do:
    #urls.append(driver.find_element_by_class_name('###').text



KeyboardInterrupt: 

### Create a data frame from the above lists

In [161]:
jobdata = pd.DataFrame(index=jobids2, data={'title':titles, 'company':companies, 'location':locations, 'salary':salaries, 'postingdate':postingdates, 'companyreview':companyreviews, 'description':descriptions})

In [162]:
#check
jobdata

,title,company,location,salary,postingdate,companyreview,description
a9749dd1260d0ccc,Traineeship: Data Analist,Createment\n24 reviews,24 reviews,"€2.500 per maand - Fulltime, Stage",8 dagen geleden,,Vraag jij je ook wel eens af hoe de apps op je...
45f4c626b11ee6dd,Data Analist,Essent\n15 reviews,15 reviews,€3.152 - €4.902 per maand,28 dagen geleden,,Bij Essent analyseert de Data Analist CX de NP...
5c387427784186c2,Data Analist,Shoeby\n33 reviews,33 reviews,,15 dagen geleden,,Details Dienstverband Fulltime Aantal uur per ...
7f8fe03c5484706b,Data-Analist,Gemeente Valkenswaard\n3 reviews,3 reviews,€4.494 per maand - Parttime,13 dagen geleden,,Gemeente Valkenswaard zoekt: Een analytische e...
8f24ece2c83af849,Data Analist,Cegeka\n28 reviews,28 reviews,,7 dagen geleden,,Als Data Analist bij Cegeka zorg jij dat data ...
...,...,...,...,...,...,...,...
5d455c047805a4aa,,,,,,,
657c5e67f432c114,,,,,,,
9ecf610c83ea2ccf,,,,,,,
2dab296d29823bd5,,,,,,,


### Save the data frame in a csv file

In [163]:
#save
jobdata.to_csv('data_analist_Nl_data.csv')

### Add the current date and time to the data set

In [167]:
# datetime object containing current date and time
now = datetime.now()

dt_string = now.strftime("%Y-%m-%d %H:%M:%S")
dt_string

'2021-03-10 20:18:19'

### Add the current date and time as a new column

In [168]:
jobdata['scrape_date'] = dt_string
#check
jobdata

,title,company,location,salary,postingdate,companyreview,description,scrape_date
a9749dd1260d0ccc,Traineeship: Data Analist,Createment\n24 reviews,24 reviews,"€2.500 per maand - Fulltime, Stage",8 dagen geleden,,Vraag jij je ook wel eens af hoe de apps op je...,2021-03-10 20:18:19
45f4c626b11ee6dd,Data Analist,Essent\n15 reviews,15 reviews,€3.152 - €4.902 per maand,28 dagen geleden,,Bij Essent analyseert de Data Analist CX de NP...,2021-03-10 20:18:19
5c387427784186c2,Data Analist,Shoeby\n33 reviews,33 reviews,,15 dagen geleden,,Details Dienstverband Fulltime Aantal uur per ...,2021-03-10 20:18:19
7f8fe03c5484706b,Data-Analist,Gemeente Valkenswaard\n3 reviews,3 reviews,€4.494 per maand - Parttime,13 dagen geleden,,Gemeente Valkenswaard zoekt: Een analytische e...,2021-03-10 20:18:19
8f24ece2c83af849,Data Analist,Cegeka\n28 reviews,28 reviews,,7 dagen geleden,,Als Data Analist bij Cegeka zorg jij dat data ...,2021-03-10 20:18:19
...,...,...,...,...,...,...,...,...
5d455c047805a4aa,,,,,,,,2021-03-10 20:18:19
657c5e67f432c114,,,,,,,,2021-03-10 20:18:19
9ecf610c83ea2ccf,,,,,,,,2021-03-10 20:18:19
2dab296d29823bd5,,,,,,,,2021-03-10 20:18:19


### Save the csv file again

In [169]:
jobdata.to_csv('data_analist_Nl_data.csv')

## To do

- Incorporate the current date and time in the scraper, rather than add it manually at the end
- If possible, tackle the Captcha